<a href="https://colab.research.google.com/github/dmejial/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Seminario/Diego_Mejia_Seminario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Seminario  
Diego Mejía Lleras  
https://github.com/dmejial/03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/Seminario  
https://colab.research.google.com/drive/1sAlYDsWwHZYmbJ1TCgTy5tk5M59KWEhs#scrollTo=RSdjSls85In5


#Solución Problema Seminario

Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las
operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/).   

Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una
cantidad dada. Un ejemplo sería para obtener el 4:
4+2-6/3*1 = 4

#1) ¿Cuántas Posibilidades hay sin tener en cuanta restricciones?

Hay (9P5)*4! formas de combinar 5 numeros extraídos de una lista de 9 números con 4 operaciones no repetidas básicas.

#2) ¿Cuál es la estructura de datos que mejor se adapta al problema?

Una tupla de números de tamaño 5 alternados por signos.

#3)¿Cuál es la función objetivo?

El valor que resulta de combinar la lista de números con los operadores. Se quiere minimizar la distancia entre este cálculo y el valor objetivo buscado.

#4) ¿Es un problema de minimización o maximización?

Se puede plantear el problema como un problema de minimizar la distancia o diferencia entre la solución que resulta de combinar la lista con los operadores matemáticos y el valor de referencia que se busca encontrar. 

#5) Algoritmo Fuerza Bruta

In [0]:
#####Costruimos una función combinatoria con argumentos (1) vector a combinar y (2) el tamaño del vector final
def xcombinations(array, n):
    if n==0: yield []
    else:
        for i in range(len(array)):
            for cc in xcombinations(array[:i]+array[i+1:],n-1):
                yield [array[i]]+cc

######Construimos una función que calcula la solución para un determinado vector y un determinado conjunto de signos operadores            

def Operaciones(array,signos,valor):
  c=[array[0]]
  for i in range(len(signos)):
      a=[eval(str(array[0])+signos[i]+str(array[1]))]
      c.append(array[1])
      array=a+array[2:]
  if array[0]==valor:
      print(array[0],c,signos)
      
#####Algoritmo de fuerza bruta: Devuelve TODAS las combinaciones posibles de vectores y signos con un valor resultante entero dado ##########

def Fuerza_Bruta(arr,signos,valor):
  for array in xcombinations(arr, 5):
    for signos in xcombinations(signos, 4):
      array=list(array)
      Operaciones(array,signos,valor)
  
#######Prueba Algoritmo####    
arr=[1,2,3,4,5,6,7,8,9]
signos=['+','-','*','/']
valor=-13

Fuerza_Bruta(arr,signos,valor)

#6) Algoritmo Genético para Solución del problema

Se implementa un algoritmo genético de solución. Se genera una población de tuplas de tamaño 5 que se cruzan y se mutan con un porcentaje de mutación predeterminado. Se especifica una función de distancia total que determina la diferencia entre la solución encontrada (el resultado de combinar la tupla con los operadores dados) y el valor que se quiere encontrar con el cálculo. La función de evaluación se establece en relación a dicha distancia. Se elige el metodo de 1-punto de corte para el cruce de padres. Se observa que tan solo con 10 generaciones ya se obtiene una solución. Se podría mejorar el algoritmo haciendo que la mutación también considerara diversas tuplas de operadores que, en este ejercicio, de dejó fijo.

In [0]:
import copy
import random 

#Se genera una solucion aleatoria de tamaño 5 a partir de los "nodos" de la lista inicial del 1 al 9
def crear_solucion(Nodos): 
  solucion=random.sample(Nodos,5)
  return solucion 

###Devuelve el valor de una solución en relación a la distancia o diferencia con el valor a obtener#####

def distancia_total(solucion,operaciones,valor):
  c=[solucion[0]]
  for i in range(len(operaciones)):
      a=[eval(str(solucion[0])+operaciones[i]+str(solucion[1]))]
      c.append(solucion[1])
      solucion=a+solucion[2:]
  #if array[0]==valor:
  return abs(solucion[0]-valor) 

#Genera una poblacion inicial de soluciones de tamaño N
def generar_poblacion(Nodos,N):
  return [crear_solucion(Nodos) for _ in range(N) ]


#Evalua la población y devuelve el mejor individuo
def Evaluar_Poblacion(poblacion,valor):
  mejor_solucion = []
  mejor_distancia = 10e100
  for p in poblacion:
    #print("solucion:", p)
    distancia_referencia = (distancia_total(p,operaciones,valor))
    if distancia_referencia < mejor_distancia:
      mejor_solucion = p
      mejor_distancia = distancia_referencia
  return mejor_solucion, mejor_distancia  


#Funcion de mutación. Se eligen dos numeros del vector y se intercambian. 
# Se hace mutaciones mutacion % de las veces
def Mutar(solucion,mutacion):
  if random.random() < mutacion:
    sel1,sel2 = sorted(random.sample(range(len(solucion)-1),2))
    return solucion[:sel1] + [solucion[sel2]] + solucion[sel1+1:sel2] + [solucion[sel1]]  + solucion[sel2+1:] 
  else:
    return solucion[::] 
  
  
  #Para el operador de cruce 1-punto los hijos generados no son soluciones(algunos nodos se repiten y otros no están)
def Factibilizar(solucion,Nodos):
  nodos_desaparecidos = list(set(Nodos) - set(solucion))
  for i in range(len(solucion) ):
    if solucion[i] in solucion[:i]:
      solucion[i] = nodos_desaparecidos.pop(0) 
  return solucion 

#Funcion para generar hijos a partir de 2 padres:
# Se elige el metodo de 1-punto de corte 
def Descendencia(padres,mutacion):
  pc = random.sample( range(1,len(padres[0])),1)[0]
  hijo1 =  Factibilizar(padres[0][:pc] + padres[1][pc:],Nodos) 
  hijo2 =  Factibilizar(padres[1][:pc] + padres[0][pc:],Nodos)
  return [hijo1,hijo2,Mutar(hijo1,mutacion),Mutar(hijo2,mutacion)]

#Funcion de cruce. Recibe una poblacion(lista de soluciones) y devuelve la población ampliada con los hijos.
def Cruzar(poblacion,mutacion):
  poblacion_copia = copy.deepcopy(poblacion)
  poblacion_final = copy.deepcopy(poblacion)

  while len(poblacion_copia) > 1: 
    padre1,padre2 = random.sample(poblacion_copia,2)
    poblacion_copia.remove(padre1)
    poblacion_copia.remove(padre2)
    poblacion_final.extend(Descendencia([padre1,padre2],mutacion))
  return poblacion_final

#Funcion de seleccion de la población. Recibe como parametro una poblacion y 
# devuelve una poblacion a la que se ha eliminado individuos poco aptos(fitness alto) y para mantener una poblacion estable de N individuos
#Se tiene en cuenta el porcentaje elitismo pasado como parametro 
def Seleccionar(poblacion, N, elitismo):
  poblacion_ordenada = sorted([ [distancia_total(solucion,operaciones,valor), solucion] for solucion in poblacion ], key= lambda x:x[0] )

  return [x[1] for x in poblacion_ordenada][:int(N*elitismo)]  +   random.sample([x[1] for x in poblacion_ordenada][int(N*elitismo):] , int(N*(1-elitismo))  )



In [0]:
#Funcion principal del algoritmo genetico
#######################################################3
def algoritmo_genetico(Nodos,signos,valor,N=100,mutacion=.15,elitismo=.1,generaciones=100):
  # N = Tamaño de la población
  # mutacion = probabilidad de una mutación
  # elitismo = porcion de la mejor poblacion a mantener
  # generaciones = nº de generaciones a generar para finalizar

  #Genera la poblacion inicial
  
  poblacion = generar_poblacion(Nodos,N)
  
  #Inicializamos valores para la mejor solucion
  (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion,valor)

  
  #Condicion de parada
  parar = False
  n=0
  #Inciamos el cliclo de generaciones
  while(parar == False) :
    
    #Cruce de la poblacion(incluye mutación)
    poblacion = Cruzar(poblacion, mutacion)
  
    #Seleccionamos la población
    poblacion = Seleccionar(poblacion, N, elitismo)
      
    #Evaluamos la nueva población
    (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion,valor)
  
    print("Generacion #", n, "\nLa mejor solución es:" , mejor_solucion, "\ncon distancia " , mejor_distancia,"para el valor:",valor)
 
    #Numero de generaciones. Criterio de parada
    if n==generaciones:
      parar = True
    n +=1
    
  return mejor_solucion

In [10]:
Nodos=[1,2,3,4,5,6,7,8,9]
operaciones=['+','-','*','/']
valor=-5
algoritmo_genetico(Nodos,operaciones,valor,N=100,mutacion=.15,elitismo=.1,generaciones=10)

Generacion # 0 
La mejor solución es: [2, 3, 6, 5, 1] 
con distancia  0.0 para el valor: -5
Generacion # 1 
La mejor solución es: [2, 3, 6, 5, 1] 
con distancia  0.0 para el valor: -5
Generacion # 2 
La mejor solución es: [2, 3, 6, 5, 1] 
con distancia  0.0 para el valor: -5
Generacion # 3 
La mejor solución es: [2, 3, 6, 5, 1] 
con distancia  0.0 para el valor: -5
Generacion # 4 
La mejor solución es: [2, 3, 6, 5, 1] 
con distancia  0.0 para el valor: -5
Generacion # 5 
La mejor solución es: [2, 3, 6, 5, 1] 
con distancia  0.0 para el valor: -5
Generacion # 6 
La mejor solución es: [2, 3, 6, 5, 1] 
con distancia  0.0 para el valor: -5
Generacion # 7 
La mejor solución es: [2, 3, 6, 5, 1] 
con distancia  0.0 para el valor: -5
Generacion # 8 
La mejor solución es: [2, 3, 6, 5, 1] 
con distancia  0.0 para el valor: -5
Generacion # 9 
La mejor solución es: [2, 3, 6, 5, 1] 
con distancia  0.0 para el valor: -5
Generacion # 10 
La mejor solución es: [2, 3, 6, 5, 1] 
con distancia  0.0 para 

[2, 3, 6, 5, 1]